In [ ]:
import numpy as np
import pandas as pd
import os

import torch 
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import time

from tqdm import tqdm

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
torch.manual_seed(0)

# Load STL 10 Dataset

In [ ]:
BATCH_SIZE = 1

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
trainset = torchvision.datasets.STL10(root="./train-data", split='train', download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [ ]:
testset = torchvision.datasets.STL10(root='./test-data', split='test', download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# AlexNet Model Feature extraction

In [ ]:
alexnet = models.alexnet(pretrained = True)
alexnet.eval()

In [ ]:
trainFeaturesList = []
trainLabels = []
for i, (inputs, labels) in tqdm(enumerate(trainloader, 0)):
    with torch.no_grad():
        feature = alexnet(inputs).detach().numpy()
    trainFeaturesList.append(feature)
    trainLabels.append(labels.item())

In [ ]:
testFeaturesList = []
testLabels = []
for i, (inputs, labels) in tqdm(enumerate(testloader, 0)):
    with torch.no_grad():
        feature = alexnet(inputs).detach().numpy()
    testFeaturesList.append(feature)
    testLabels.append(labels.item())

In [ ]:
trainFeatures = np.array(trainFeaturesList).reshape(len(trainFeaturesList), trainFeaturesList[0].shape[0] * trainFeaturesList[0].shape[1])
print(trainFeatures.shape)

In [ ]:
testFeatures = np.array(testFeaturesList).reshape(len(testFeaturesList), testFeaturesList[0].shape[0] * testFeaturesList[0].shape[1])
print(testFeatures.shape)

# Model training and predictions 

## Logistic Regression

In [ ]:
lrmodel = LogisticRegression(solver = 'saga', max_iter = 5000)
lrmodel.fit(trainFeatures, trainLabels)

In [ ]:
y_pred = lrmodel.predict(testFeatures)
acc = accuracy_score(y_pred, testLabels)
print(acc)

## Linear SVC

In [ ]:
clf = LinearSVC(max_iter = 10000)
clf.fit(trainFeatures, trainLabels)

In [ ]:
y_pred = clf.predict(testFeatures)
acc = accuracy_score(y_pred, testLabels)
print(acc)

## Gaussian NB

In [ ]:
gnb = GaussianNB()
gnb.fit(trainFeatures, trainLabels)

In [ ]:
y_pred = gnb.predict(testFeatures)
acc = accuracy_score(y_pred, testLabels)
print(acc)